In [42]:
from Utils_MSA import *
from importlib import reload
    
rcParams = {'font.size': 24, 'font.weight': 'normal', 'font.family': 'sans-serif',
            'axes.unicode_minus':False, 'axes.labelweight':'normal'}
plt.rcParams.update(rcParams)

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [43]:
filedir = '/Users/harihara/Mount-2/hotspring_metagenome/single_cell_analysis_with_Gabe_Birzu/16S_Read_Alignments/'
samples = listdir(filedir+'Alpha_Alleles/paf_files/')

fdir = '/Users/harihara/Mount-2/hotspring_metagenome/single_cell_analysis_with_Gabe_Birzu/16S_Gene_Segments/'
out = {}
for f in ['Synechococcus_Alpha_16SrRNA','Synechococcus_Beta_16SrRNA','Synechococcus_Gamma_16SrRNA']:
    lines = open(fdir+f+'.fna').readlines()
    s = ''
    for l in lines:
        if not l.startswith('>'):
            s += l.replace("\n","")
    out[f.replace("Synechococcus_","").replace("_16SrRNA","")] = s    

In [44]:
start = 350
end = 550
MSA = {'Alpha':{'Alpha_Ref':out['Alpha'][start:end]}, 'Beta':{'Beta_Ref':out['Beta'][start:end]}, 
       'Gamma':{'Gamma_Ref':out['Gamma'][start:end]}}

MSA_duplicates = {'Alpha':dict({}), 'Beta':dict({}), 'Gamma':dict({})}
df_op = pd.DataFrame()
retain_duplicates = True

for s in samples:
    #if 'MS65' not in s:
    #    continue
    print(s)
    
    df_alpha = Load_PAF(filedir+'Alpha_Alleles/paf_files/'+s, s)
    df_beta = Load_PAF(filedir+'Beta_Alleles/paf_files/'+s, s)
    df_gamma = Load_PAF(filedir+'Gamma_Alleles/paf_files/'+s, s)
    
    Temp = pd.DataFrame()
    Temp = Temp.append(df_alpha).append(df_beta).append(df_gamma)
    df_op = df_op.append(Temp, ignore_index = True)
    
    D = Create_MSA(Temp, start, end, s, out)
    if retain_duplicates == False:
        for a in MSA.keys():
            MSA[a].update(D[a])
    else:
        for a in D.keys():
            for o in D[a]:
                if D[a][o] not in MSA_duplicates[a]:
                    MSA[a][o] = D[a][o]
                    MSA_duplicates[a][D[a][o]] = [o]
                else:
                    MSA_duplicates[a][D[a][o]].append(o)  
            print(a, len(MSA_duplicates[a]), len(MSA[a]))
          

HotsprSampleMS50_FD.paf
17571 17748
19536 19719
19979 20165
Alpha 9 10
Beta 152 153
Gamma 1 2
HotsprottomLayer_2_FD.paf
7149 7252
7335 7438
8467 8593
Alpha 14 15
Beta 171 172
Gamma 9 10
Hotspr2Sample149_FD.paf
10804 10886
11164 11233
12798 12885
Alpha 27 28
Beta 217 218
HotsprSampleR4cd_FD.paf
59982 60294
67855 68183
67866 68189
Alpha 59 60
Beta 465 466
Gamma 14 15
HotsprSampOS1265_FD.paf
22906 23054
22267 22417
25771 25922
Alpha 216 217
Beta 466 467
Gamma 48 49
HotsprSampleOS50_FD.paf
11269 11334
12458 12527
12514 12584
Alpha 222 223
Beta 520 521
Gamma 66 67
HotsprSampleOS60_FD.paf
43652 43942
42546 42815
48703 48996
Alpha 431 432
Beta 526 527
Gamma 74 75
HotsprOSTMatCore_FD.paf
3008 3056
3432 3476
3972 4020
Alpha 432 433
Beta 530 531
Hotspr20SampleT9_FD.paf
33960 34180
35013 35236
38544 38764
Alpha 495 496
Beta 616 617
HotsprSampleMSe2_FD.paf
26729 26879
29333 29499
30345 30498
Alpha 506 507
Beta 728 729
Gamma 76 77
HotsprSampleMSe3_FD.paf
21130 21287
22502 22659
24196 24360
Alpha 52

In [45]:
alpha = MSA['Alpha']['Alpha_Ref']
beta = MSA['Beta']['Beta_Ref']
gamma = MSA['Gamma']['Gamma_Ref']

del MSA['Alpha']['Alpha_Ref'],MSA['Beta']['Beta_Ref'],MSA['Gamma']['Gamma_Ref']

In [46]:
d_alpha_beta, tracker_alpha_beta = Edit(alpha, beta)
Align_alpha, Align_beta = Return_Alignments(tracker_alpha_beta, alpha, beta)

d_alpha_beta_gamma, tracker_alpha_beta_gamma = Edit(Align_alpha, gamma) 
Align_alpha_gamma, Align_gamma = Return_Alignments(tracker_alpha_beta_gamma, Align_alpha, gamma, '+')
Align_beta_gamma = Merge_MSA(Align_alpha_gamma, {'Beta':Align_beta}, '+')['Beta']

Alpha_backbone = Align_alpha_gamma.replace("+","-")
Beta_backbone = Align_beta_gamma.replace("+","-")
Gamma_backbone = Align_gamma.replace("+","-")

assert ((len(Alpha_backbone) == len(Beta_backbone)) & 
        (len(Gamma_backbone) == len(Beta_backbone))), "Length Mismatch"

In [47]:
MSA_alpha = Merge_MSA(Alpha_backbone, MSA['Alpha'], '-')
MSA_beta = Merge_MSA(Beta_backbone, MSA['Beta'], '-')
MSA_gamma = Merge_MSA(Gamma_backbone, MSA['Gamma'], '-')

MSA_All = MSA_alpha.copy()
MSA_All.update(MSA_beta)
MSA_All.update(MSA_gamma)

MSA_All['Alpha'] = Alpha_backbone
MSA_All['Beta'] = Beta_backbone
MSA_All['Gamma'] = Gamma_backbone

print(len(MSA_All))

4003


In [48]:
o = open('/Users/harihara/Mount-2/hotspring_metagenome/single_cell_analysis_with_Gabe_Birzu/\
16S_Read_Alignments/MSA_350_550.fasta','w')
for M in MSA_All:
    o.write(">"+M+"\n")
    o.write(MSA_All[M]+"\n")
o.close()

In [49]:
o = open('/Users/harihara/Mount-2/hotspring_metagenome/single_cell_analysis_with_Gabe_Birzu/\
16S_Read_Alignments/MSA_350_550_Duplicates.dict','w')
o.write(str(MSA_duplicates))
o.close()